# teste dos dados importados

In [18]:
import numpy as np
import pandas as pd

In [19]:
# Ler o arquivo Excel para um DataFrame
dados = pd.read_excel('./dados_especialista.xlsx')
 

In [20]:
dados.dtypes

FEBRE         int64
MIALGIA       int64
CEFALEIA      int64
EXANTEMA      int64
NAUSEA        int64
DOR_COSTAS    int64
CONJUNTVIT    int64
ARTRITE       int64
ARTRALGIA     int64
PETEQUIA_N    int64
DOR_RETRO     int64
DIABETES      int64
HIPERTENSA    int64
DIAS          int64
CLASSI_FIN    int64
dtype: object

# separação e construção do dataset

In [21]:
dengue_df = dados[dados['CLASSI_FIN'] == 1.0]
chikungunya_df = dados[dados['CLASSI_FIN'] == 2.0]
outras_doencas_df = dados[dados['CLASSI_FIN'] == 0.0]

In [22]:
contagem_classes = dados['CLASSI_FIN'].value_counts()

# Para imprimir a contagem de cada classe
print(contagem_classes)

print(dados.shape)

CLASSI_FIN
0    75344
1    75344
2    75344
Name: count, dtype: int64
(226032, 15)


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split

In [24]:
 

# Divisão em treino e teste
dengue_train, dengue_test = train_test_split(dengue_df, test_size=0.3, random_state=42)
chikungunya_train, chikungunya_test = train_test_split(chikungunya_df, test_size=0.3, random_state=42)
outras_doencas_train, outras_doencas_test = train_test_split(outras_doencas_df, test_size=0.3, random_state=42)

 

# Conjunto de treino e teste binário 
binary_train = pd.concat([dengue_train, outras_doencas_train])
binary_test = pd.concat([dengue_test, outras_doencas_test])

# conjunto de treino e teste multiclasse
multiclasse_train = pd.concat([dengue_train, outras_doencas_train, chikungunya_train])
multiclasse_teste = pd.concat([dengue_test, outras_doencas_test, chikungunya_test])


# Preparação dos dados para treino 
x_binary_train_numpy = binary_train.drop(columns="CLASSI_FIN", axis=1).values
y_binary_train_numpy = binary_train["CLASSI_FIN"].values

# Preparação dos dados para teste
x_binary_test_numpy = binary_test.drop(columns="CLASSI_FIN", axis=1).values
y_binary_test_numpy = binary_test["CLASSI_FIN"].values


# Preparação dos dados para treino 
x_multiclasse_train_numpy = multiclasse_train.drop(columns="CLASSI_FIN", axis=1).values
y_multiclasse_train_numpy = multiclasse_train["CLASSI_FIN"].values

# Preparação dos dados para teste
x_multiclasse_teste_numpy = multiclasse_teste.drop(columns="CLASSI_FIN", axis=1).values
y_multiclasse_teste_numpy = multiclasse_teste["CLASSI_FIN"].values

## contagem

In [25]:
contagem_classes = binary_train['CLASSI_FIN'].value_counts()

# Para imprimir a contagem de cada classe
print(contagem_classes)
print(binary_train.shape)
 

CLASSI_FIN
1    52740
0    52740
Name: count, dtype: int64
(105480, 15)


In [26]:
contagem_classes = binary_test['CLASSI_FIN'].value_counts()

# Para imprimir a contagem de cada classe
print(contagem_classes)
print(binary_test.shape)
 

CLASSI_FIN
1    22604
0    22604
Name: count, dtype: int64
(45208, 15)


In [27]:
contagem_classes = multiclasse_train['CLASSI_FIN'].value_counts()

# Para imprimir a contagem de cada classe
print(contagem_classes)

print(multiclasse_train.shape)

CLASSI_FIN
1    52740
0    52740
2    52740
Name: count, dtype: int64
(158220, 15)


In [28]:
contagem_classes = multiclasse_teste['CLASSI_FIN'].value_counts()

# Para imprimir a contagem de cada classe
print(contagem_classes)

print(multiclasse_teste.shape)

CLASSI_FIN
1    22604
0    22604
2    22604
Name: count, dtype: int64
(67812, 15)


# Otimização de algoritmos

## tipo binario

In [29]:
arvore = DecisionTreeClassifier(random_state=42)

param_grid = {
   'max_depth':range(1, 200, 1),
   "criterion":["gini","entropy"],
   "max_features": [None, "sqrt", "log2"]
}

grid_search = GridSearchCV(
    estimator=arvore,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10,
    verbose=4,
    n_jobs=-1,
)

grid_search.fit(x_binary_train_numpy, y_binary_train_numpy)

best_params = grid_search.best_params_
best_params


Fitting 10 folds for each of 1194 candidates, totalling 11940 fits


{'criterion': 'entropy', 'max_depth': 12, 'max_features': None}

In [ ]:
floresta_aleatoria = RandomForestClassifier(random_state=42)

param_grid = {
   "criterion":["gini","entropy"],
   "n_estimators":range(10, 250, 10), 
   "min_samples_leaf": [1, 2, 3, 4, 5]
}

grid_search = GridSearchCV(
    estimator=floresta_aleatoria,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10,
    verbose=4,
    n_jobs=-1,
)

grid_search.fit(x_binary_train_numpy, y_binary_train_numpy)

best_params = grid_search.best_params_
best_params


In [30]:
# Após o treinamento do GridSearchCV
# Acessar os resultados do GridSearchCV
results = pd.DataFrame(grid_search.cv_results_)

# Ordenar os resultados pela média da pontuação de validação cruzada (mean_test_score)
# e selecionar as 3 melhores combinações de parâmetros
top_3_results = results.sort_values(by='mean_test_score', ascending=False).head(3)

# Mostrar os 3 melhores resultados
results.sort_values(by='mean_test_score', ascending=False).head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
630,0.383390,0.015972,0.005470,0.001237,entropy,12,None,"{'criterion': 'entropy', 'max_depth': 12, 'max...",0.685533,0.678328,...,0.669890,0.673682,0.670838,0.670174,0.679465,0.677948,0.670554,0.675540,0.005019,1
621,0.375341,0.016824,0.004844,0.000791,entropy,9,None,"{'criterion': 'entropy', 'max_depth': 9, 'max_...",0.684300,0.678138,...,0.668942,0.674630,0.674441,0.670743,0.675673,0.672924,0.671976,0.675095,0.004277,2
33,0.340624,0.015548,0.004844,0.000514,gini,12,None,"{'criterion': 'gini', 'max_depth': 12, 'max_fe...",0.684680,0.675578,...,0.667994,0.672545,0.672071,0.671122,0.677474,0.679276,0.667994,0.674659,0.005032,3


## tipo multiclasse

In [ ]:
arvore = DecisionTreeClassifier(random_state=42)

param_grid = {
   'max_depth':range(1, 200, 1),
   "criterion":["gini","entropy"],
   "max_features": [None, "sqrt", "log2"]
}

grid_search = GridSearchCV(
    estimator=arvore,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10,
    verbose=4,
    n_jobs=-1,
)

grid_search.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)

best_params = grid_search.best_params_
best_params


In [ ]:
floresta_aleatoria = RandomForestClassifier(random_state=42)

param_grid = {
   "criterion":["gini","entropy"],
   "n_estimators":range(10, 250, 10), 
   "min_samples_leaf": [1, 2, 3, 4, 5]
}

grid_search = GridSearchCV(
    estimator=floresta_aleatoria,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10,
    verbose=4,
    n_jobs=-1,
)

grid_search.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)

best_params = grid_search.best_params_
best_params


In [ ]:
# Após o treinamento do GridSearchCV
# Acessar os resultados do GridSearchCV
results = pd.DataFrame(grid_search.cv_results_)

# Ordenar os resultados pela média da pontuação de validação cruzada (mean_test_score)
# e selecionar as 3 melhores combinações de parâmetros
top_3_results = results.sort_values(by='mean_test_score', ascending=False).head(3)

# Mostrar os 3 melhores resultados
results.sort_values(by='mean_test_score', ascending=False).head(3)

# treinamento e avaliação dos algortitmos

## tipo binario

In [31]:



# Treinamento do modelo
arvore_avaliacao = DecisionTreeClassifier(random_state=42, criterion="entropy", max_depth=12, max_features=None)
arvore_avaliacao.fit(x_binary_train_numpy, y_binary_train_numpy)

# Predição e avaliação
y_teste_previsao = arvore_avaliacao.predict(x_binary_test_numpy)
metricas_teste = classification_report(y_binary_test_numpy, y_teste_previsao, output_dict=True)

metricas_teste


{'0': {'precision': 0.6637418266188568,
  'recall': 0.6960714917713678,
  'f1-score': 0.6795223390701591,
  'support': 22604.0},
 '1': {'precision': 0.6805096963214435,
  'recall': 0.6473632985312334,
  'f1-score': 0.6635227968349695,
  'support': 22604.0},
 'accuracy': 0.6717173951513007,
 'macro avg': {'precision': 0.6721257614701501,
  'recall': 0.6717173951513007,
  'f1-score': 0.6715225679525643,
  'support': 45208.0},
 'weighted avg': {'precision': 0.6721257614701501,
  'recall': 0.6717173951513007,
  'f1-score': 0.6715225679525644,
  'support': 45208.0}}

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_binary_test_numpy, y_teste_previsao)

In [ ]:
# TESTE COM DADOS DE TREINO
# Treinamento do modelo
arvore_avaliacao = DecisionTreeClassifier(random_state=42, criterion="entropy", max_depth=12, max_features=None)
arvore_avaliacao.fit(x_binary_train_numpy, y_binary_train_numpy)

# Predição e avaliação
y_treino_previsao = arvore_avaliacao.predict(x_binary_train_numpy)
metricas_treino = classification_report(y_binary_train_numpy, y_treino_previsao, output_dict=True)

metricas_treino

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_binary_train_numpy, y_treino_previsao)

In [ ]:
# desempenho com dados de teste
floresta_aleatoria_avaliacao = RandomForestClassifier(random_state=42, criterion="gini", min_samples_leaf=5, n_estimators=240)
floresta_aleatoria_avaliacao.fit(x_binary_train_numpy, y_binary_train_numpy)

# Predição e avaliação
y_teste_previsao = floresta_aleatoria_avaliacao.predict(x_binary_test_numpy)
metricas_teste = classification_report(y_binary_test_numpy, y_teste_previsao, output_dict=True)

metricas_teste


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_binary_test_numpy, y_teste_previsao)

In [ ]:
# desempenho com dados de treino
floresta_aleatoria_avaliacao =  RandomForestClassifier(random_state=42, criterion="gini", min_samples_leaf=5, n_estimators=240)
floresta_aleatoria_avaliacao.fit(x_binary_train_numpy, y_binary_train_numpy)

# Predição e avaliação
y_treino_previsao = floresta_aleatoria_avaliacao.predict(x_binary_train_numpy)
metricas_treino = classification_report(y_binary_train_numpy, y_treino_previsao, output_dict=True)

metricas_treino


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_binary_train_numpy, y_treino_previsao)

## tipo multiclasse

In [32]:
# arvore de decisao - desempenho teste
arvore_avaliacao = DecisionTreeClassifier(random_state=42, criterion="gini", max_depth=10, max_features=None)
arvore_avaliacao.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)

# Predição e avaliação
y_teste_previsao = arvore_avaliacao.predict(x_multiclasse_teste_numpy)
metricas_teste = classification_report(y_multiclasse_teste_numpy, y_teste_previsao, output_dict=True)

metricas_teste


{'0': {'precision': 0.469287680660702,
  'recall': 0.4826579366483808,
  'f1-score': 0.4758789147692576,
  'support': 22604.0},
 '1': {'precision': 0.5797455968688845,
  'recall': 0.5766678463988675,
  'f1-score': 0.5782026259758695,
  'support': 22604.0},
 '2': {'precision': 0.5722373188405797,
  'recall': 0.5589718633870111,
  'f1-score': 0.565526810491451,
  'support': 22604.0},
 'accuracy': 0.5394325488114198,
 'macro avg': {'precision': 0.5404235321233887,
  'recall': 0.5394325488114198,
  'f1-score': 0.5398694504121927,
  'support': 67812.0},
 'weighted avg': {'precision': 0.5404235321233888,
  'recall': 0.5394325488114198,
  'f1-score': 0.5398694504121927,
  'support': 67812.0}}

In [ ]:
# Treinamento do modelo
arvore_avaliacao = DecisionTreeClassifier(random_state=42, criterion="gini", max_depth=10, max_features=None)
arvore_avaliacao.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)

# Predição e avaliação
y_treino_previsao = arvore_avaliacao.predict(x_multiclasse_train_numpy)
metricas_treino = classification_report(y_multiclasse_train_numpy, y_treino_previsao, output_dict=True)

metricas_treino


In [ ]:
# floresta aleatoria - teste
floresta_aleatoria_avaliacao = RandomForestClassifier(random_state=42, criterion="entropy", min_samples_leaf=5, n_estimators=170)
floresta_aleatoria_avaliacao.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)

# Predição e avaliação
y_teste_previsao = floresta_aleatoria_avaliacao.predict(x_multiclasse_teste_numpy)
metricas_teste = classification_report(y_multiclasse_teste_numpy, y_teste_previsao, output_dict=True)

metricas_teste


In [ ]:
# floresta aleatoria - treino
floresta_aleatoria_avaliacao = RandomForestClassifier(random_state=42, criterion="entropy", min_samples_leaf=5, n_estimators=170)
floresta_aleatoria_avaliacao.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)

# Predição e avaliação
y_treino_previsao = floresta_aleatoria_avaliacao.predict(x_multiclasse_train_numpy)
metricas_treino = classification_report(y_multiclasse_train_numpy, y_treino_previsao, output_dict=True)

metricas_treino


## matriz de confusão  

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix( y_multiclasse_teste_numpy, y_teste_previsao)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix( y_multiclasse_train_numpy, y_treino_previsao)

# validação cruzada de Monte Carlo

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:
metrica_teste = {
     "arvore_binaria": {
          "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []  
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            }
         }
     },
     "arvore_multiclasse": {
         "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "2": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },    
         }

     },
     "floresta_binaria": {
          "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []  
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            }
         }
     },
     "floresta_multiclasse": {
         "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "2": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },    
         }

     }
}

In [14]:
metrica_treino = {
     "arvore_binaria": {
          "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []  
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            }
         }
     },
     "arvore_multiclasse": {
         "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "2": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },    
         }

     },
     "floresta_binaria": {
          "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []  
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            }
         }
     },
     "floresta_multiclasse": {
         "macro": {
            "acuracia": [],
            "precision": [],
            "recall": [],
            "f1-score": [],
            "support": []
         },
         "classes": {
            "0": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "1": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },
            "2": {
                "precision": [],
                "recall": [],
                "f1-score": [],
                "support": []
            },    
         }

     }
}

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

for i in range(100):
    # Separação dos dados por classe
    dengue_df = dados[dados['CLASSI_FIN'] == 1]
    chikungunya_df = dados[dados['CLASSI_FIN'] == 2]
    outras_doencas_df = dados[dados['CLASSI_FIN'] == 0]

    # Divisão em treino e teste
    dengue_train, dengue_test = train_test_split(dengue_df, test_size=0.3, random_state=i)
    chikungunya_train, chikungunya_test = train_test_split(chikungunya_df, test_size=0.3, random_state=i)
    outras_doencas_train, outras_doencas_test = train_test_split(outras_doencas_df, test_size=0.3, random_state=i)

    # Conjunto de treino e teste binário 
    binary_train = pd.concat([dengue_train, outras_doencas_train])
    binary_test = pd.concat([dengue_test, outras_doencas_test])
    multiclasse_train = pd.concat([dengue_train, outras_doencas_train, chikungunya_train])
    multiclasse_teste = pd.concat([dengue_test, outras_doencas_test, chikungunya_test])

    # Preparação dos dados para treino 
    x_binary_train_numpy = binary_train.drop(columns="CLASSI_FIN", axis=1).values
    y_binary_train_numpy = binary_train["CLASSI_FIN"].values

    # Preparação dos dados para teste
    x_binary_test_numpy = binary_test.drop(columns="CLASSI_FIN", axis=1).values
    y_binary_test_numpy = binary_test["CLASSI_FIN"].values

    # Preparação dos dados para treino 
    x_multiclasse_train_numpy = multiclasse_train.drop(columns="CLASSI_FIN", axis=1).values
    y_multiclasse_train_numpy = multiclasse_train["CLASSI_FIN"].values

    # Preparação dos dados para teste
    x_multiclasse_teste_numpy = multiclasse_teste.drop(columns="CLASSI_FIN", axis=1).values
    y_multiclasse_teste_numpy = multiclasse_teste["CLASSI_FIN"].values

    # definição de modelos
    modelo_binario_arvore = DecisionTreeClassifier(random_state=i, criterion="gini", max_depth=11, max_features=None)
    modelo_binario_floresta = RandomForestClassifier(random_state=i, criterion="entropy", min_samples_leaf=5, n_estimators=220)
    
    modelo_multiclasse_arvore = DecisionTreeClassifier(random_state=i, criterion="gini", max_depth=12, max_features=None)
    modelo_multiclasse_floresta =  RandomForestClassifier(random_state=i, criterion="entropy", min_samples_leaf=5, n_estimators=230)

    # treinamento dos modelos
    modelo_binario_arvore.fit(x_binary_train_numpy, y_binary_train_numpy)
    modelo_binario_floresta.fit(x_binary_train_numpy, y_binary_train_numpy)
    modelo_multiclasse_arvore.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)
    modelo_multiclasse_floresta.fit(x_multiclasse_train_numpy, y_multiclasse_train_numpy)

    
    # avaliação teste
    y_previsao_modelo_binario_arvore = modelo_binario_arvore.predict(x_binary_test_numpy)
    y_previsao_modelo_binario_floresta = modelo_binario_floresta.predict(x_binary_test_numpy)
    y_previsao_modelo_multiclasse_arvore = modelo_multiclasse_arvore.predict(x_multiclasse_teste_numpy)
    y_previsao_modelo_multiclasse_floresta = modelo_multiclasse_floresta.predict(x_multiclasse_teste_numpy)

    # metricas teste binario
    metricas_teste_binario_arvore = classification_report(y_binary_test_numpy, y_previsao_modelo_binario_arvore, output_dict=True)
    metrica_teste['arvore_binaria']['macro']['acuracia'].append(metricas_teste_binario_arvore['accuracy'])
    metrica_teste['arvore_binaria']['macro']['f1-score'].append(metricas_teste_binario_arvore['macro avg']['f1-score'])
    metrica_teste['arvore_binaria']['macro']['precision'].append(metricas_teste_binario_arvore['macro avg']['precision'])
    metrica_teste['arvore_binaria']['macro']['recall'].append(metricas_teste_binario_arvore['macro avg']['recall'])
    metrica_teste['arvore_binaria']['macro']['support'].append(metricas_teste_binario_arvore['macro avg']['support'])
    for classe in ['0', '1']:
        metrica_teste['arvore_binaria']['classes'][classe]['precision'].append(metricas_teste_binario_arvore[classe]['precision'])
        metrica_teste['arvore_binaria']['classes'][classe]['recall'].append(metricas_teste_binario_arvore[classe]['recall'])
        metrica_teste['arvore_binaria']['classes'][classe]['f1-score'].append(metricas_teste_binario_arvore[classe]['f1-score'])
        metrica_teste['arvore_binaria']['classes'][classe]['support'].append(metricas_teste_binario_arvore[classe]['support']) 

    metricas_teste_binario_floresta = classification_report(y_binary_test_numpy, y_previsao_modelo_binario_floresta, output_dict=True)
    metrica_teste['floresta_binaria']['macro']['acuracia'].append(metricas_teste_binario_floresta['accuracy'])
    metrica_teste['floresta_binaria']['macro']['f1-score'].append(metricas_teste_binario_floresta['macro avg']['f1-score'])
    metrica_teste['floresta_binaria']['macro']['precision'].append(metricas_teste_binario_floresta['macro avg']['precision'])
    metrica_teste['floresta_binaria']['macro']['recall'].append(metricas_teste_binario_floresta['macro avg']['recall'])
    metrica_teste['floresta_binaria']['macro']['support'].append(metricas_teste_binario_floresta['macro avg']['support'])
    for classe in ['0', '1']:
        metrica_teste['floresta_binaria']['classes'][classe]['precision'].append(metricas_teste_binario_floresta[classe]['precision'])
        metrica_teste['floresta_binaria']['classes'][classe]['recall'].append(metricas_teste_binario_floresta[classe]['recall'])
        metrica_teste['floresta_binaria']['classes'][classe]['f1-score'].append(metricas_teste_binario_floresta[classe]['f1-score'])
        metrica_teste['floresta_binaria']['classes'][classe]['support'].append(metricas_teste_binario_floresta[classe]['support']) 

    ## metricas teste multiclasse
    metricas_teste_multiclasse_arvore = classification_report(y_multiclasse_teste_numpy, y_previsao_modelo_multiclasse_arvore, output_dict=True)
    metrica_teste['arvore_multiclasse']['macro']['acuracia'].append(metricas_teste_multiclasse_arvore['accuracy'])
    metrica_teste['arvore_multiclasse']['macro']['f1-score'].append(metricas_teste_multiclasse_arvore['macro avg']['f1-score'])
    metrica_teste['arvore_multiclasse']['macro']['precision'].append(metricas_teste_multiclasse_arvore['macro avg']['precision'])
    metrica_teste['arvore_multiclasse']['macro']['recall'].append(metricas_teste_multiclasse_arvore['macro avg']['recall'])
    metrica_teste['arvore_multiclasse']['macro']['support'].append(metricas_teste_multiclasse_arvore['macro avg']['support'])
    for classe in ['0', '1', '2']:
        metrica_teste['arvore_multiclasse']['classes'][classe]['precision'].append(metricas_teste_multiclasse_arvore[classe]['precision'])
        metrica_teste['arvore_multiclasse']['classes'][classe]['recall'].append(metricas_teste_multiclasse_arvore[classe]['recall'])
        metrica_teste['arvore_multiclasse']['classes'][classe]['f1-score'].append(metricas_teste_multiclasse_arvore[classe]['f1-score'])
        metrica_teste['arvore_multiclasse']['classes'][classe]['support'].append(metricas_teste_multiclasse_arvore[classe]['support']) 

    metricas_teste_multiclasse_floresta = classification_report(y_multiclasse_teste_numpy, y_previsao_modelo_multiclasse_floresta, output_dict=True)
    metrica_teste['floresta_multiclasse']['macro']['acuracia'].append(metricas_teste_multiclasse_floresta['accuracy'])
    metrica_teste['floresta_multiclasse']['macro']['f1-score'].append(metricas_teste_multiclasse_floresta['macro avg']['f1-score'])
    metrica_teste['floresta_multiclasse']['macro']['precision'].append(metricas_teste_multiclasse_floresta['macro avg']['precision'])
    metrica_teste['floresta_multiclasse']['macro']['recall'].append(metricas_teste_multiclasse_floresta['macro avg']['recall'])
    metrica_teste['floresta_multiclasse']['macro']['support'].append(metricas_teste_multiclasse_floresta['macro avg']['support'])
    for classe in ['0', '1', '2']:
        metrica_teste['floresta_multiclasse']['classes'][classe]['precision'].append(metricas_teste_multiclasse_floresta[classe]['precision'])
        metrica_teste['floresta_multiclasse']['classes'][classe]['recall'].append(metricas_teste_multiclasse_floresta[classe]['recall'])
        metrica_teste['floresta_multiclasse']['classes'][classe]['f1-score'].append(metricas_teste_multiclasse_floresta[classe]['f1-score'])
        metrica_teste['floresta_multiclasse']['classes'][classe]['support'].append(metricas_teste_multiclasse_floresta[classe]['support'])
    
    ##################################################################################

    # avaliação treino
    y_previsao_modelo_binario_arvore_treino = modelo_binario_arvore.predict(x_binary_train_numpy)
    y_previsao_modelo_binario_floresta_treino = modelo_binario_floresta.predict(x_binary_train_numpy)
    y_previsao_modelo_multiclasse_arvore_treino = modelo_multiclasse_arvore.predict(x_multiclasse_train_numpy)
    y_previsao_modelo_multiclasse_floresta_treino = modelo_multiclasse_floresta.predict(x_multiclasse_train_numpy)

    # metricas teste binario
    metricas_treino_binario_arvore = classification_report(y_binary_train_numpy, y_previsao_modelo_binario_arvore_treino, output_dict=True)
    metrica_treino['arvore_binaria']['macro']['acuracia'].append(metricas_treino_binario_arvore['accuracy'])
    metrica_treino['arvore_binaria']['macro']['f1-score'].append(metricas_treino_binario_arvore['macro avg']['f1-score'])
    metrica_treino['arvore_binaria']['macro']['precision'].append(metricas_treino_binario_arvore['macro avg']['precision'])
    metrica_treino['arvore_binaria']['macro']['recall'].append(metricas_treino_binario_arvore['macro avg']['recall'])
    metrica_treino['arvore_binaria']['macro']['support'].append(metricas_treino_binario_arvore['macro avg']['support'])
    for classe in ['0', '1']:
        metrica_treino['arvore_binaria']['classes'][classe]['precision'].append(metricas_treino_binario_arvore[classe]['precision'])
        metrica_treino['arvore_binaria']['classes'][classe]['recall'].append(metricas_treino_binario_arvore[classe]['recall'])
        metrica_treino['arvore_binaria']['classes'][classe]['f1-score'].append(metricas_treino_binario_arvore[classe]['f1-score'])
        metrica_treino['arvore_binaria']['classes'][classe]['support'].append(metricas_treino_binario_arvore[classe]['support']) 

    metricas_treino_binario_floresta = classification_report(y_binary_train_numpy, y_previsao_modelo_binario_floresta_treino, output_dict=True)
    metrica_treino['floresta_binaria']['macro']['acuracia'].append(metricas_treino_binario_floresta['accuracy'])
    metrica_treino['floresta_binaria']['macro']['f1-score'].append(metricas_treino_binario_floresta['macro avg']['f1-score'])
    metrica_treino['floresta_binaria']['macro']['precision'].append(metricas_treino_binario_floresta['macro avg']['precision'])
    metrica_treino['floresta_binaria']['macro']['recall'].append(metricas_treino_binario_floresta['macro avg']['recall'])
    metrica_treino['floresta_binaria']['macro']['support'].append(metricas_treino_binario_floresta['macro avg']['support'])
    for classe in ['0', '1']:
        metrica_treino['floresta_binaria']['classes'][classe]['precision'].append(metricas_treino_binario_floresta[classe]['precision'])
        metrica_treino['floresta_binaria']['classes'][classe]['recall'].append(metricas_treino_binario_floresta[classe]['recall'])
        metrica_treino['floresta_binaria']['classes'][classe]['f1-score'].append(metricas_treino_binario_floresta[classe]['f1-score'])
        metrica_treino['floresta_binaria']['classes'][classe]['support'].append(metricas_treino_binario_floresta[classe]['support']) 

    ## metricas teste multiclasse
    metricas_treino_multiclasse_arvore = classification_report(y_multiclasse_train_numpy, y_previsao_modelo_multiclasse_arvore_treino, output_dict=True)
    metrica_treino['arvore_multiclasse']['macro']['acuracia'].append(metricas_treino_multiclasse_arvore['accuracy'])
    metrica_treino['arvore_multiclasse']['macro']['f1-score'].append(metricas_treino_multiclasse_arvore['macro avg']['f1-score'])
    metrica_treino['arvore_multiclasse']['macro']['precision'].append(metricas_treino_multiclasse_arvore['macro avg']['precision'])
    metrica_treino['arvore_multiclasse']['macro']['recall'].append(metricas_treino_multiclasse_arvore['macro avg']['recall'])
    metrica_treino['arvore_multiclasse']['macro']['support'].append(metricas_treino_multiclasse_arvore['macro avg']['support'])
    for classe in ['0', '1', '2']:
        metrica_treino['arvore_multiclasse']['classes'][classe]['precision'].append(metricas_treino_multiclasse_arvore[classe]['precision'])
        metrica_treino['arvore_multiclasse']['classes'][classe]['recall'].append(metricas_treino_multiclasse_arvore[classe]['recall'])
        metrica_treino['arvore_multiclasse']['classes'][classe]['f1-score'].append(metricas_treino_multiclasse_arvore[classe]['f1-score'])
        metrica_treino['arvore_multiclasse']['classes'][classe]['support'].append(metricas_treino_multiclasse_arvore[classe]['support']) 

    metricas_treino_multiclasse_floresta = classification_report(y_multiclasse_train_numpy, y_previsao_modelo_multiclasse_floresta_treino, output_dict=True)
    metrica_treino['floresta_multiclasse']['macro']['acuracia'].append(metricas_treino_multiclasse_floresta['accuracy'])
    metrica_treino['floresta_multiclasse']['macro']['f1-score'].append(metricas_treino_multiclasse_floresta['macro avg']['f1-score'])
    metrica_treino['floresta_multiclasse']['macro']['precision'].append(metricas_treino_multiclasse_floresta['macro avg']['precision'])
    metrica_treino['floresta_multiclasse']['macro']['recall'].append(metricas_treino_multiclasse_floresta['macro avg']['recall'])
    metrica_treino['floresta_multiclasse']['macro']['support'].append(metricas_treino_multiclasse_floresta['macro avg']['support'])
    for classe in ['0', '1', '2']:
        metrica_treino['floresta_multiclasse']['classes'][classe]['precision'].append(metricas_treino_multiclasse_floresta[classe]['precision'])
        metrica_treino['floresta_multiclasse']['classes'][classe]['recall'].append(metricas_treino_multiclasse_floresta[classe]['recall'])
        metrica_treino['floresta_multiclasse']['classes'][classe]['f1-score'].append(metricas_treino_multiclasse_floresta[classe]['f1-score'])
        metrica_treino['floresta_multiclasse']['classes'][classe]['support'].append(metricas_treino_multiclasse_floresta[classe]['support'])
    

    
    

In [16]:
import numpy as np

In [ ]:
# verificação se executou 100 vezes
for modelo, metricas in metrica_teste.items():
    print(f"Modelo: {modelo}")
    
    # Tamanho das métricas macro
    print("Métricas Macro:")
    for chave, valor in metricas["macro"].items():
        print(f"  {chave}: {len(valor)}")

    # Tamanho das métricas por classe
    print("Métricas por Classe:")
    for classe, metrics in metricas["classes"].items():
        print(f"  Classe: {classe}")
        for chave, valor in metrics.items():
            print(f"    {chave}: {len(valor)}")


In [ ]:
# verificação se executou 100 vezes
for modelo, metricas in metrica_teste.items():
    print(f"Modelo: {modelo}")
    
    # Tamanho das métricas macro
    print("Métricas Macro:")
    for chave, valor in metricas["macro"].items():
        print(f"  {chave}: {np.sum(valor)/100}")

    # Tamanho das métricas por classe
    print("Métricas por Classe:")
    for classe, metrics in metricas["classes"].items():
        print(f"  Classe: {classe}")
        for chave, valor in metrics.items():
            print(f"    {chave}: {np.sum(valor)/100}")
    
    print("")


### Arvore de decisão - tipo binario

In [ ]:
print("Arvore de decisão - tipo binario - avaliação teste: ")
print("acuracia: ", np.mean(metrica_teste['arvore_binaria']['macro']['acuracia']))
print("precision: ", np.mean(metrica_teste['arvore_binaria']['macro']['precision']))
print("recall: ", np.mean(metrica_teste['arvore_binaria']['macro']['recall']))
print("f1-score: ", np.mean(metrica_teste['arvore_binaria']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_teste['arvore_binaria']['macro']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_teste['arvore_binaria']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_teste['arvore_binaria']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_teste['arvore_binaria']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_teste['arvore_binaria']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_teste['arvore_binaria']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_teste['arvore_binaria']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_teste['arvore_binaria']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_teste['arvore_binaria']['classes']['0']['support']))

In [ ]:
print("Arvore de decisão - tipo binario - avaliação treino: ")
print("acuracia: ", np.mean(metrica_treino['arvore_binaria']['macro']['acuracia']))
print("precision: ", np.mean(metrica_treino['arvore_binaria']['macro']['precision']))
print("recall: ", np.mean(metrica_treino['arvore_binaria']['macro']['recall']))
print("f1-score: ", np.mean(metrica_treino['arvore_binaria']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_treino['arvore_binaria']['macro']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_treino['arvore_binaria']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_treino['arvore_binaria']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_treino['arvore_binaria']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_treino['arvore_binaria']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_treino['arvore_binaria']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_treino['arvore_binaria']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_treino['arvore_binaria']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_treino['arvore_binaria']['classes']['0']['support']))



### Floresta Aleatoria - tipo binario

In [ ]:
print("Floresta Aleatoria - tipo binario - avaliação teste: ")
print("acuracia: ", np.mean(metrica_teste['floresta_binaria']['macro']['acuracia']))
print("precision: ", np.mean(metrica_teste['floresta_binaria']['macro']['precision']))
print("recall: ", np.mean(metrica_teste['floresta_binaria']['macro']['recall']))
print("f1-score: ", np.mean(metrica_teste['floresta_binaria']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_teste['floresta_binaria']['macro']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_teste['floresta_binaria']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_teste['floresta_binaria']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_teste['floresta_binaria']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_teste['floresta_binaria']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_teste['floresta_binaria']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_teste['floresta_binaria']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_teste['floresta_binaria']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_teste['floresta_binaria']['classes']['0']['support']))

In [ ]:
print("Floresta Aleatoria - tipo binario - avaliação treino: ")
print("acuracia: ", np.mean(metrica_treino['floresta_binaria']['macro']['acuracia']))
print("precision: ", np.mean(metrica_treino['floresta_binaria']['macro']['precision']))
print("recall: ", np.mean(metrica_treino['floresta_binaria']['macro']['recall']))
print("f1-score: ", np.mean(metrica_treino['floresta_binaria']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_treino['floresta_binaria']['macro']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_treino['floresta_binaria']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_treino['floresta_binaria']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_treino['floresta_binaria']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_treino['floresta_binaria']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_treino['floresta_binaria']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_treino['floresta_binaria']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_treino['floresta_binaria']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_treino['floresta_binaria']['classes']['0']['support']))

### Arvore de decisão - tipo multiclasse

In [ ]:
print("Arvore de decisão - tipo multiclasse - avaliação teste: ")
print("acuracia: ", np.mean(metrica_teste['arvore_multiclasse']['macro']['acuracia']))
print("precision: ", np.mean(metrica_teste['arvore_multiclasse']['macro']['precision']))
print("recall: ", np.mean(metrica_teste['arvore_multiclasse']['macro']['recall']))
print("f1-score: ", np.mean(metrica_teste['arvore_multiclasse']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_teste['arvore_multiclasse']['macro']['support']))

print("\n")
print("classe CHIKUNGUNYA (2): ")
print("precision: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['2']['precision']))
print("recall: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['2']['recall']))
print("f1-score: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['2']['f1-score']))
print("support: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['2']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_teste['arvore_multiclasse']['classes']['0']['support']))

In [ ]:
print("Arvore de decisão - tipo multiclasse - avaliação treino: ")
print("acuracia: ", np.mean(metrica_treino['arvore_multiclasse']['macro']['acuracia']))
print("precision: ", np.mean(metrica_treino['arvore_multiclasse']['macro']['precision']))
print("recall: ", np.mean(metrica_treino['arvore_multiclasse']['macro']['recall']))
print("f1-score: ", np.mean(metrica_treino['arvore_multiclasse']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_treino['arvore_multiclasse']['macro']['support']))

print("\n")
print("classe CHIKUNGUNYA (2): ")
print("precision: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['2']['precision']))
print("recall: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['2']['recall']))
print("f1-score: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['2']['f1-score']))
print("support: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['2']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_treino['arvore_multiclasse']['classes']['0']['support']))

### Floresta Aleatoria - tipo multiclasse

In [ ]:
print("Floresta Aleatoria - tipo multiclasse - avaliação teste: ")
print("acuracia: ", np.mean(metrica_teste['floresta_multiclasse']['macro']['acuracia']))
print("precision: ", np.mean(metrica_teste['floresta_multiclasse']['macro']['precision']))
print("recall: ", np.mean(metrica_teste['floresta_multiclasse']['macro']['recall']))
print("f1-score: ", np.mean(metrica_teste['floresta_multiclasse']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_teste['floresta_multiclasse']['macro']['support']))

print("\n")
print("classe CHIKUNGUNYA (2): ")
print("precision: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['2']['precision']))
print("recall: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['2']['recall']))
print("f1-score: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['2']['f1-score']))
print("support: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['2']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_teste['floresta_multiclasse']['classes']['0']['support']))

In [ ]:
print("Floresta Aleatoria - tipo multiclasse - avaliação treino: ")
print("acuracia: ", np.mean(metrica_treino['floresta_multiclasse']['macro']['acuracia']))
print("precision: ", np.mean(metrica_treino['floresta_multiclasse']['macro']['precision']))
print("recall: ", np.mean(metrica_treino['floresta_multiclasse']['macro']['recall']))
print("f1-score: ", np.mean(metrica_treino['floresta_multiclasse']['macro']['f1-score']))
print("suporte: ", np.mean(metrica_treino['floresta_multiclasse']['macro']['support']))

print("\n")
print("classe CHIKUNGUNYA (2): ")
print("precision: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['2']['precision']))
print("recall: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['2']['recall']))
print("f1-score: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['2']['f1-score']))
print("support: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['2']['support']))

print("\n")
print("classe DENGUE (1): ")
print("precision: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['1']['precision']))
print("recall: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['1']['recall']))
print("f1-score: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['1']['f1-score']))
print("support: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['1']['support']))

print("\n")
print("classe OUTRAS DOENCAS (0): ")
print("precision: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['0']['precision']))
print("recall: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['0']['recall']))
print("f1-score: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['0']['f1-score']))
print("support: ", np.mean(metrica_treino['floresta_multiclasse']['classes']['0']['support']))

# RESULTADOS PARA TESTE ESTATISTICO

## ARVORE DE DECISÃO BINARIA

In [32]:
#  MACRO
acuracia_macro = metrica_teste['arvore_binaria']['macro']['acuracia']
precision_macro = metrica_teste['arvore_binaria']['macro']['precision']
f1score_macro = metrica_teste['arvore_binaria']['macro']['f1-score']
recall_macro = metrica_teste['arvore_binaria']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_teste['arvore_binaria']['classes']['DENGUE']['precision']
f1score_1 = metrica_teste['arvore_binaria']['classes']['DENGUE']['f1-score']
recall_1 = metrica_teste['arvore_binaria']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_teste['arvore_binaria']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_teste['arvore_binaria']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_teste['arvore_binaria']['classes']['OUTRAS_DOENCAS']['recall']

In [34]:
#  MACRO
acuracia_macro = metrica_treino['arvore_binaria']['macro']['acuracia']
precision_macro = metrica_treino['arvore_binaria']['macro']['precision']
f1score_macro = metrica_treino['arvore_binaria']['macro']['f1-score']
recall_macro = metrica_treino['arvore_binaria']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_treino['arvore_binaria']['classes']['DENGUE']['precision']
f1score_1 = metrica_treino['arvore_binaria']['classes']['DENGUE']['f1-score']
recall_1 = metrica_treino['arvore_binaria']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_treino['arvore_binaria']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_treino['arvore_binaria']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_treino['arvore_binaria']['classes']['OUTRAS_DOENCAS']['recall']

In [ ]:
 
print("MACRO")
print("\n")
print("acuracia:")
print(acuracia_macro)
print("\n")
print("precision:")
print(precision_macro)
print("\n")
print("recall:")
print(recall_macro)
print("\n")
print("f1-score:")
print(f1score_macro)

print("\n")
print("CLASSES:")
print("\n")
print("DENGUE 1:")
print("\n")
print("precision:")
print(precision_1)
print("\n")
print("f1-score:")
print(f1score_1)
print("\n")
print("recall:")
print(recall_1)

print("\n")
print("OUTRAS DOENÇAS 0:")
print("\n")
print("precision:")
print(precision_0)
print("\n")
print("recall:")
print(recall_0)
print("\n")
print("f1-score:")
print(f1score_0)

 

## FLORESTA ALEATORIA BINARIA

In [36]:
#  MACRO
acuracia_macro = metrica_teste['floresta_binaria']['macro']['acuracia']
precision_macro = metrica_teste['floresta_binaria']['macro']['precision']
f1score_macro = metrica_teste['floresta_binaria']['macro']['f1-score']
recall_macro = metrica_teste['floresta_binaria']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_teste['floresta_binaria']['classes']['DENGUE']['precision']
f1score_1 = metrica_teste['floresta_binaria']['classes']['DENGUE']['f1-score']
recall_1 = metrica_teste['floresta_binaria']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_teste['floresta_binaria']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_teste['floresta_binaria']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_teste['floresta_binaria']['classes']['OUTRAS_DOENCAS']['recall']

In [38]:
#  MACRO
acuracia_macro = metrica_treino['floresta_binaria']['macro']['acuracia']
precision_macro = metrica_treino['floresta_binaria']['macro']['precision']
f1score_macro = metrica_treino['floresta_binaria']['macro']['f1-score']
recall_macro = metrica_treino['floresta_binaria']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_treino['floresta_binaria']['classes']['DENGUE']['precision']
f1score_1 = metrica_treino['floresta_binaria']['classes']['DENGUE']['f1-score']
recall_1 = metrica_treino['floresta_binaria']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_treino['floresta_binaria']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_treino['floresta_binaria']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_treino['floresta_binaria']['classes']['OUTRAS_DOENCAS']['recall']

In [ ]:
 
print("MACRO")
print("\n")
print("acuracia:")
print(acuracia_macro)
print("\n")
print("precision:")
print(precision_macro)
print("\n")
print("recall:")
print(recall_macro)
print("\n")
print("f1-score:")
print(f1score_macro)

print("\n")
print("CLASSES:")
print("\n")
print("DENGUE 1:")
print("\n")
print("precision:")
print(precision_1)
print("\n")
print("f1-score:")
print(f1score_1)
print("\n")
print("recall:")
print(recall_1)

print("\n")
print("OUTRAS DOENÇAS 0:")
print("\n")
print("precision:")
print(precision_0)
print("\n")
print("recall:")
print(recall_0)
print("\n")
print("f1-score:")
print(f1score_0)

 

## ARVORE DE DECISÃO MULTICLASSE

In [40]:
#  MACRO
acuracia_macro = metrica_teste['arvore_multiclasse']['macro']['acuracia']
precision_macro = metrica_teste['arvore_multiclasse']['macro']['precision']
f1score_macro = metrica_teste['arvore_multiclasse']['macro']['f1-score']
recall_macro = metrica_teste['arvore_multiclasse']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_teste['arvore_multiclasse']['classes']['DENGUE']['precision']
f1score_1 = metrica_teste['arvore_multiclasse']['classes']['DENGUE']['f1-score']
recall_1 = metrica_teste['arvore_multiclasse']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_teste['arvore_multiclasse']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_teste['arvore_multiclasse']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_teste['arvore_multiclasse']['classes']['OUTRAS_DOENCAS']['recall']

# CLASSE 1 - CHIKUNGUNYA
precision_2 = metrica_teste['arvore_multiclasse']['classes']['CHIKUNGUNYA']['precision']
f1score_2 = metrica_teste['arvore_multiclasse']['classes']['CHIKUNGUNYA']['f1-score']
recall_2 = metrica_teste['arvore_multiclasse']['classes']['CHIKUNGUNYA']['recall']

In [42]:
#  MACRO
acuracia_macro = metrica_treino['arvore_multiclasse']['macro']['acuracia']
precision_macro = metrica_treino['arvore_multiclasse']['macro']['precision']
f1score_macro = metrica_treino['arvore_multiclasse']['macro']['f1-score']
recall_macro = metrica_treino['arvore_multiclasse']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_treino['arvore_multiclasse']['classes']['DENGUE']['precision']
f1score_1 = metrica_treino['arvore_multiclasse']['classes']['DENGUE']['f1-score']
recall_1 = metrica_treino['arvore_multiclasse']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_treino['arvore_multiclasse']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_treino['arvore_multiclasse']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_treino['arvore_multiclasse']['classes']['OUTRAS_DOENCAS']['recall']

# CLASSE 1 - CHIKUNGUNYA
precision_2 = metrica_treino['arvore_multiclasse']['classes']['CHIKUNGUNYA']['precision']
f1score_2 = metrica_treino['arvore_multiclasse']['classes']['CHIKUNGUNYA']['f1-score']
recall_2 = metrica_treino['arvore_multiclasse']['classes']['CHIKUNGUNYA']['recall']

In [ ]:
 
print("MACRO")
print("\n")
print("acuracia:")
print(acuracia_macro)
print("\n")
print("precision:")
print(precision_macro)
print("\n")
print("recall:")
print(recall_macro)
print("\n")
print("f1-score:")
print(f1score_macro)

print("\n")
print("CLASSES:")
print("\n")

print("DENGUE 1:")
print("\n")
print("precision:")
print(precision_1)
print("\n")
print("f1-score:")
print(f1score_1)
print("\n")
print("recall:")
print(recall_1)

print("\n")
print("OUTRAS DOENÇAS 0:")
print("\n")
print("precision:")
print(precision_0)
print("\n")
print("recall:")
print(recall_0)
print("\n")
print("f1-score:")
print(f1score_0)

print("\n")
print("CHIKUNGUNYA 2:")
print("\n")
print("precision:")
print(precision_2)
print("\n")
print("f1-score:")
print(f1score_2)
print("\n")
print("recall:")
print(recall_2)
 

## FLORESTA ALEATORIA MULTICLASSE

In [44]:
#  MACRO
acuracia_macro = metrica_teste['floresta_multiclasse']['macro']['acuracia']
precision_macro = metrica_teste['floresta_multiclasse']['macro']['precision']
f1score_macro = metrica_teste['floresta_multiclasse']['macro']['f1-score']
recall_macro = metrica_teste['floresta_multiclasse']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_teste['floresta_multiclasse']['classes']['DENGUE']['precision']
f1score_1 = metrica_teste['floresta_multiclasse']['classes']['DENGUE']['f1-score']
recall_1 = metrica_teste['floresta_multiclasse']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_teste['floresta_multiclasse']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_teste['floresta_multiclasse']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_teste['floresta_multiclasse']['classes']['OUTRAS_DOENCAS']['recall']

# CLASSE 1 - CHIKUNGUNYA
precision_2 = metrica_teste['floresta_multiclasse']['classes']['CHIKUNGUNYA']['precision']
f1score_2 = metrica_teste['floresta_multiclasse']['classes']['CHIKUNGUNYA']['f1-score']
recall_2 = metrica_teste['floresta_multiclasse']['classes']['CHIKUNGUNYA']['recall']

In [46]:
#  MACRO
acuracia_macro = metrica_treino['floresta_multiclasse']['macro']['acuracia']
precision_macro = metrica_treino['floresta_multiclasse']['macro']['precision']
f1score_macro = metrica_treino['floresta_multiclasse']['macro']['f1-score']
recall_macro = metrica_treino['floresta_multiclasse']['macro']['recall']

# CLASSE 1 - DENGUE
precision_1 = metrica_treino['floresta_multiclasse']['classes']['DENGUE']['precision']
f1score_1 = metrica_treino['floresta_multiclasse']['classes']['DENGUE']['f1-score']
recall_1 = metrica_treino['floresta_multiclasse']['classes']['DENGUE']['recall']

# CLASSE 0 - OUTRAS DOENCAS
precision_0 = metrica_treino['floresta_multiclasse']['classes']['OUTRAS_DOENCAS']['precision']
f1score_0 = metrica_treino['floresta_multiclasse']['classes']['OUTRAS_DOENCAS']['f1-score']
recall_0 = metrica_treino['floresta_multiclasse']['classes']['OUTRAS_DOENCAS']['recall']

# CLASSE 1 - CHIKUNGUNYA
precision_2 = metrica_treino['floresta_multiclasse']['classes']['CHIKUNGUNYA']['precision']
f1score_2 = metrica_treino['floresta_multiclasse']['classes']['CHIKUNGUNYA']['f1-score']
recall_2 = metrica_treino['floresta_multiclasse']['classes']['CHIKUNGUNYA']['recall']

In [ ]:
 
print("MACRO")
print("\n")
print("acuracia:")
print(acuracia_macro)
print("\n")
print("precision:")
print(precision_macro)
print("\n")
print("recall:")
print(recall_macro)
print("\n")
print("f1-score:")
print(f1score_macro)

print("\n")
print("CLASSES:")
print("\n")

print("DENGUE 1:")
print("\n")
print("precision:")
print(precision_1)
print("\n")
print("f1-score:")
print(f1score_1)
print("\n")
print("recall:")
print(recall_1)

print("\n")
print("OUTRAS DOENÇAS 0:")
print("\n")
print("precision:")
print(precision_0)
print("\n")
print("recall:")
print(recall_0)
print("\n")
print("f1-score:")
print(f1score_0)

print("\n")
print("CHIKUNGUNYA 2:")
print("\n")
print("precision:")
print(precision_2)
print("\n")
print("f1-score:")
print(f1score_2)
print("\n")
print("recall:")
print(recall_2)
 